In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from sklearn import metrics
import re
from sklearn import tree
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.tree import export_graphviz
from collections import OrderedDict
!pip install XlsxWriter
import xlsxwriter
from subprocess import call
import csv
import joblib
from sklearn.feature_selection import SelectFromModel

In [ ]:
df = pd.read_csv("csv_result-merged.csv", error_bad_lines=False)
df.head()

,id,file_name,corpus_name,sheet_name,sheet_index,table_name,cell_address,cell_type=0,cell_type=2,length,words,leading_spaces,first_char_num,first_char_special,capitalized,all_upper,is_alpha,special_chars,punctuations,contains_colon,words_like_total,words_like_table,in_year_range,is_aggr_formula=1,ref_val_type=0,first_row_num,first_col_num,num_of_neighbors=0,num_of_neighbors=1,num_of_neighbors=2,num_of_neighbors=3,num_of_neighbors=4,h_alignment=0,h_alignment=2,v_alignment=2,indentation,fill_patern=0,is_wraptext,num_of_cells,border_top_type=0,border_top_type=1,border_bottom_type=0,border_left_type=0,border_right_type=0,border_right_type=2,cell_borders=0,font_height,is_font_color_default,is_bold,underline_type=0,label,matches_top_style,matches_bottom_style,matches_left_style,matches_right_style,matches_top_type,matches_bottom_type,matches_left_type,matches_right_type,top_neighbor_type,bottom_neighbor_type,left_neighbor_type,right_neighbor_type
0,1,michelle_lokay__26590__IGSUpdate.xls,ENRON,Sheet1,0,NaN,A1,f,t,16,3,0,f,f,f,f,f,f,f,f,f,f,f,f,f,0,0,t,f,f,f,f,t,f,t,0,t,f,1,f,f,f,t,t,f,f,12,f,f,t,metadata,9,9,9,9,9,9,9,9,9,9,9,9
1,2,michelle_lokay__26590__IGSUpdate.xls,ENRON,Sheet1,0,NaN,I1,f,t,28,0,0,f,f,f,f,f,f,f,f,f,f,f,f,f,0,8,t,f,f,f,f,f,f,t,0,t,f,1,f,f,f,t,t,f,f,12,f,f,t,metadata,9,9,9,9,9,9,9,9,9,9,9,9
2,3,michelle_lokay__26590__IGSUpdate.xls,ENRON,Sheet1,0,NaN,D8:H8,f,t,28,0,0,f,f,f,f,f,f,f,f,f,f,f,f,f,7,3,t,f,f,f,f,f,t,t,0,t,f,5,f,f,f,f,f,f,f,12,f,t,t,metadata,9,9,9,9,9,9,9,9,9,9,9,9
3,4,michelle_lokay__26590__IGSUpdate.xls,ENRON,Sheet1,0,SHEET1_ANNOTATION_TABLE_B11_J28,C11,f,t,34,4,0,f,f,f,f,f,f,t,f,f,f,f,f,f,10,2,f,t,f,f,f,f,f,t,0,t,f,1,t,f,t,t,t,f,t,10,f,t,f,metadata,9,1,9,9,9,1,9,9,9,2,9,9
4,5,michelle_lokay__26590__IGSUpdate.xls,ENRON,Sheet1,0,SHEET1_ANNOTATION_TABLE_B11_J28,C12,f,t,24,7,0,f,f,f,f,f,f,t,f,f,f,f,f,f,11,2,f,t,f,f,f,f,f,t,0,t,f,1,t,f,t,t,t,f,t,10,f,t,f,metadata,1,9,9,9,1,9,9,9,2,9,9,9


In [ ]:
df.columns

Index(['id', 'file_name', 'corpus_name', 'sheet_name', 'sheet_index',
       'table_name', 'cell_address', 'cell_type=0', 'cell_type=2', 'length',
       'words', 'leading_spaces', 'first_char_num', 'first_char_special',
       'capitalized', 'all_upper', 'is_alpha', 'special_chars', 'punctuations',
       'contains_colon', 'words_like_total', 'words_like_table',
       'in_year_range', 'is_aggr_formula=1', 'ref_val_type=0', 'first_row_num',
       'first_col_num', 'num_of_neighbors=0', 'num_of_neighbors=1',
       'num_of_neighbors=2', 'num_of_neighbors=3', 'num_of_neighbors=4',
       'h_alignment=0', 'h_alignment=2', 'v_alignment=2', 'indentation',
       'fill_patern=0', 'is_wraptext', 'num_of_cells', 'border_top_type=0',
       'border_top_type=1', 'border_bottom_type=0', 'border_left_type=0',
       'border_right_type=0', 'border_right_type=2', 'cell_borders=0',
       'font_height', 'is_font_color_default', 'is_bold', 'underline_type=0',
       'label', 'matches_top_style', 'mat

In [ ]:
columns_to_drop = ["id", "file_name", "table_name", "corpus_name", "sheet_name", "sheet_index"]
df = df.drop(columns_to_drop, axis=1)
df.head()

,cell_address,cell_type=0,cell_type=2,length,words,leading_spaces,first_char_num,first_char_special,capitalized,all_upper,is_alpha,special_chars,punctuations,contains_colon,words_like_total,words_like_table,in_year_range,is_aggr_formula=1,ref_val_type=0,first_row_num,first_col_num,num_of_neighbors=0,num_of_neighbors=1,num_of_neighbors=2,num_of_neighbors=3,num_of_neighbors=4,h_alignment=0,h_alignment=2,v_alignment=2,indentation,fill_patern=0,is_wraptext,num_of_cells,border_top_type=0,border_top_type=1,border_bottom_type=0,border_left_type=0,border_right_type=0,border_right_type=2,cell_borders=0,font_height,is_font_color_default,is_bold,underline_type=0,label,matches_top_style,matches_bottom_style,matches_left_style,matches_right_style,matches_top_type,matches_bottom_type,matches_left_type,matches_right_type,top_neighbor_type,bottom_neighbor_type,left_neighbor_type,right_neighbor_type
0,A1,f,t,16,3,0,f,f,f,f,f,f,f,f,f,f,f,f,f,0,0,t,f,f,f,f,t,f,t,0,t,f,1,f,f,f,t,t,f,f,12,f,f,t,metadata,9,9,9,9,9,9,9,9,9,9,9,9
1,I1,f,t,28,0,0,f,f,f,f,f,f,f,f,f,f,f,f,f,0,8,t,f,f,f,f,f,f,t,0,t,f,1,f,f,f,t,t,f,f,12,f,f,t,metadata,9,9,9,9,9,9,9,9,9,9,9,9
2,D8:H8,f,t,28,0,0,f,f,f,f,f,f,f,f,f,f,f,f,f,7,3,t,f,f,f,f,f,t,t,0,t,f,5,f,f,f,f,f,f,f,12,f,t,t,metadata,9,9,9,9,9,9,9,9,9,9,9,9
3,C11,f,t,34,4,0,f,f,f,f,f,f,t,f,f,f,f,f,f,10,2,f,t,f,f,f,f,f,t,0,t,f,1,t,f,t,t,t,f,t,10,f,t,f,metadata,9,1,9,9,9,1,9,9,9,2,9,9
4,C12,f,t,24,7,0,f,f,f,f,f,f,t,f,f,f,f,f,f,11,2,f,t,f,f,f,f,f,t,0,t,f,1,t,f,t,t,t,f,t,10,f,t,f,metadata,1,9,9,9,1,9,9,9,2,9,9,9


In [ ]:
df.columns


Index(['cell_address', 'cell_type=0', 'cell_type=2', 'length', 'words',
       'leading_spaces', 'first_char_num', 'first_char_special', 'capitalized',
       'all_upper', 'is_alpha', 'special_chars', 'punctuations',
       'contains_colon', 'words_like_total', 'words_like_table',
       'in_year_range', 'is_aggr_formula=1', 'ref_val_type=0', 'first_row_num',
       'first_col_num', 'num_of_neighbors=0', 'num_of_neighbors=1',
       'num_of_neighbors=2', 'num_of_neighbors=3', 'num_of_neighbors=4',
       'h_alignment=0', 'h_alignment=2', 'v_alignment=2', 'indentation',
       'fill_patern=0', 'is_wraptext', 'num_of_cells', 'border_top_type=0',
       'border_top_type=1', 'border_bottom_type=0', 'border_left_type=0',
       'border_right_type=0', 'border_right_type=2', 'cell_borders=0',
       'font_height', 'is_font_color_default', 'is_bold', 'underline_type=0',
       'label', 'matches_top_style', 'matches_bottom_style',
       'matches_left_style', 'matches_right_style', 'matches_top

In [ ]:
 categorical_cols = ['cell_type=0', 'cell_type=2', 'first_char_num', 'first_char_special', 'capitalized', 
    'all_upper', 'is_alpha', 'special_chars', 'punctuations', 'contains_colon', 'words_like_total', 
    'words_like_table', 'in_year_range', 'is_aggr_formula=1', 'ref_val_type=0', 'num_of_neighbors=0', 
    'num_of_neighbors=1', 'num_of_neighbors=2', 'num_of_neighbors=3', 'num_of_neighbors=4', 'h_alignment=0', 
    'h_alignment=2', 'v_alignment=2', 'fill_patern=0', 'is_wraptext', 'border_top_type=0', 
    'border_top_type=1', 'border_bottom_type=0', 'border_left_type=0', 'border_right_type=0', 
    'border_right_type=2', 'cell_borders=0', 'is_font_color_default', 'is_bold', 'underline_type=0']
for colName in categorical_cols:
  df[colName] = df[colName].astype('category')
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
df.head()

,cell_address,cell_type=0,cell_type=2,length,words,leading_spaces,first_char_num,first_char_special,capitalized,all_upper,is_alpha,special_chars,punctuations,contains_colon,words_like_total,words_like_table,in_year_range,is_aggr_formula=1,ref_val_type=0,first_row_num,first_col_num,num_of_neighbors=0,num_of_neighbors=1,num_of_neighbors=2,num_of_neighbors=3,num_of_neighbors=4,h_alignment=0,h_alignment=2,v_alignment=2,indentation,fill_patern=0,is_wraptext,num_of_cells,border_top_type=0,border_top_type=1,border_bottom_type=0,border_left_type=0,border_right_type=0,border_right_type=2,cell_borders=0,font_height,is_font_color_default,is_bold,underline_type=0,label,matches_top_style,matches_bottom_style,matches_left_style,matches_right_style,matches_top_type,matches_bottom_type,matches_left_type,matches_right_type,top_neighbor_type,bottom_neighbor_type,left_neighbor_type,right_neighbor_type
0,A1,0,1,16,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,1,1,0,0,12,0,0,1,metadata,9,9,9,9,9,9,9,9,9,9,9,9
1,I1,0,1,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,1,0,0,12,0,0,1,metadata,9,9,9,9,9,9,9,9,9,9,9,9
2,D8:H8,0,1,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,3,1,0,0,0,0,0,1,1,0,1,0,5,0,0,0,0,0,0,0,12,0,1,1,metadata,9,9,9,9,9,9,9,9,9,9,9,9
3,C11,0,1,34,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,10,2,0,1,0,0,0,0,0,1,0,1,0,1,1,0,1,1,1,0,1,10,0,1,0,metadata,9,1,9,9,9,1,9,9,9,2,9,9
4,C12,0,1,24,7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,11,2,0,1,0,0,0,0,0,1,0,1,0,1,1,0,1,1,1,0,1,10,0,1,0,metadata,1,9,9,9,1,9,9,9,2,9,9,9


In [ ]:
clf = RandomForestClassifier(bootstrap=True,
 criterion= 'entropy',
 max_depth= 15,
 max_features= 'auto',
 n_estimators= 500)


In [ ]:
df_full = df
y = df_full['label'].values
X = df_full.drop(['label'], axis=1)
X = X.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
df_full.columns

Index(['startColumnNumber', 'startRowNumber', 'endColumnNumber',
       'endRowNumber', 'cell_type=0', 'cell_type=2', 'length', 'words',
       'leading_spaces', 'first_char_num', 'first_char_special', 'capitalized',
       'all_upper', 'is_alpha', 'special_chars', 'punctuations',
       'contains_colon', 'words_like_total', 'words_like_table',
       'in_year_range', 'is_aggr_formula=1', 'ref_val_type=0', 'first_row_num',
       'first_col_num', 'num_of_neighbors=0', 'num_of_neighbors=1',
       'num_of_neighbors=2', 'num_of_neighbors=3', 'num_of_neighbors=4',
       'h_alignment=0', 'h_alignment=2', 'v_alignment=2', 'indentation',
       'fill_patern=0', 'is_wraptext', 'num_of_cells', 'border_top_type=0',
       'border_top_type=1', 'border_bottom_type=0', 'border_left_type=0',
       'border_right_type=0', 'border_right_type=2', 'cell_borders=0',
       'font_height', 'is_font_color_default', 'is_bold', 'underline_type=0',
       'label', 'matches_top_style', 'matches_bottom_style',

In [ ]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
sel = SelectFromModel(RandomForestClassifier(bootstrap=True,
 criterion= 'entropy',
 max_depth= 15,
 max_features= 'auto',
 n_estimators= 500))
# sel.fit(X_train, y_train)
# sel.get_support()

In [ ]:
 y_full_predict = []
 j = 0
 for row in X_test:
    prediction = clf.predict([row])
    y_full_predict.append(prediction)
    j+=1
    
print(y_full_predict)
print(y_test)

[array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['header'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['header'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['header'], dtype=object), array(['data'], dtype=object), array(['header'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['header'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['header'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['derived'], dtype=object), array(['attributes'], dtype=object), array(['derived'], dtype=object), array(['data'], dtype=object), array(['header'], dtype=object), array(['data'], dtype=object), array(['data

In [ ]:
confusion_mtx = metrics.confusion_matrix(y_test, y_full_predict, labels = ['data', 'header', 'attributes', 'metadata', 'derived'])
print(confusion_mtx)
print(metrics.classification_report(y_test, y_full_predict, labels=['data', 'header', 'attributes', 'metadata', 'derived']))

[[6593   13   24    7    6]
 [  41 1886    8    3    0]
 [  25    5  796    3    0]
 [  27   14    0  601    0]
 [  32    0    0    0  488]]
              precision    recall  f1-score   support

        data       0.98      0.99      0.99      6643
      header       0.98      0.97      0.98      1938
  attributes       0.96      0.96      0.96       829
    metadata       0.98      0.94      0.96       642
     derived       0.99      0.94      0.96       520

    accuracy                           0.98     10572
   macro avg       0.98      0.96      0.97     10572
weighted avg       0.98      0.98      0.98     10572



In [ ]:
filename = 'finalized_model_full_80.sav'
joblib.dump(clf, filename)

['finalized_model_full_80.sav']

In [ ]:
df_full = df
y = df_full['label'].values
X = df_full.drop(['label'], axis=1)
X = X.to_numpy()

In [ ]:
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
filename = 'finalized_model_full.sav'
joblib.dump(clf, filename)

['finalized_model_full.sav']

In [ ]:
y_pred_five_arr = []
y_test_five_arr = []
cross_val = StratifiedKFold(n_splits=10)
def fiveClassClassifier(df):
  df_three = df
  Y = df_three['label'].values
  X = df_three.drop(['label'], axis=1)
  header = X.columns.to_numpy()
  i = 0
  for train_index, test_index in cross_val.split(X, Y):
    X_train = X.to_numpy()[train_index]
    y_train = Y[train_index]
    X_test = X.to_numpy()[test_index]
    y_test = Y[test_index]
    clf.fit(X_train, y_train)
    filename = 'finalized_model_five_'+str(i)+'.sav'
    joblib.dump(clf, filename)
   

    y_predicted = []
    
    for row in X_test:
      prediction = clf.predict([row])
      y_predicted.append(prediction)
      
    print(y_predicted)
    print(y_test)

    y_pred_five_arr.append(y_predicted)
    y_test_five_arr.append(y_test)

    i += 1

In [ ]:
fiveClassClassifier(df)

[array(['metadata'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['header'], dtype=object), array(['data'], dtype=object), array(['metadata'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data']

In [ ]:
def printMetrics(y_pred_arr, y_test_arr, target_names):
  combined = [[0 for j in range(len(target_names))] for i in range (len(target_names))]
  print(combined)

  for i in range(10):
    count = 0
    y_predicted = y_pred_arr[i]
    y_test = y_test_arr[i]
    for j in range(len(y_predicted)):
      pred = y_predicted[j][0]
      if(pred != y_test[j]):
        count += 1

    print("Fold:", i+1)

    count = Counter(y_test)
    for target in target_names:
      print(target, ":", count[target])
    confusion_mtx = metrics.confusion_matrix(y_test, y_predicted, labels = target_names)
    combined = [[combined[i][j] + confusion_mtx[i][j]  for j in range(len(target_names))] for i in range(len(target_names))]
    print(confusion_mtx)
    print(metrics.classification_report(y_test, y_predicted, labels=target_names))
    print("------------")


  for i in range(len(combined)):
    print(combined[i])
  recall = np.diag(combined) / np.sum(combined, axis = 1)
  precision = np.diag(combined) / np.sum(combined, axis = 0)
  f1Score = []
  for i in range(5):
    f1Score.append((2*recall[i]*precision[i])/(recall[i] + precision[i]))
    
  print("precision", "recall", "f1Score")
  for i in range(len(recall)) :
      print(target_names[i], ":", precision[i], recall[i], f1Score[i])


In [ ]:
printMetrics(y_pred_five_arr, y_test_five_arr,  ['data', 'header', 'attributes', 'metadata', 'derived'])

[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
Fold: 1
data : 3281
header : 978
attributes : 422
metadata : 336
derived : 269
[[3116   74   36   29   26]
 [  68  806   72   32    0]
 [ 171   19  216   16    0]
 [  76   97   16  145    2]
 [  96    0    0    0  173]]
              precision    recall  f1-score   support

        data       0.88      0.95      0.92      3281
      header       0.81      0.82      0.82       978
  attributes       0.64      0.51      0.57       422
    metadata       0.65      0.43      0.52       336
     derived       0.86      0.64      0.74       269

    accuracy                           0.84      5286
   macro avg       0.77      0.67      0.71      5286
weighted avg       0.83      0.84      0.84      5286

------------
Fold: 2
data : 3281
header : 978
attributes : 422
metadata : 336
derived : 269
[[3141   49   40   15   36]
 [ 216  723    5   31    3]
 [  50    0  369    3    0]
 [  58   55   37  186    0]
 